## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Bredasdorp,-34.5322,20.0403,58.28,77,100,4.90,ZA,overcast clouds
1,1,Nikolskoye,59.7035,30.7861,69.69,75,44,5.10,RU,scattered clouds
2,2,Ushuaia,-54.8000,-68.3000,35.22,86,0,5.75,AR,clear sky
3,3,Irtyshskiy,58.3333,68.1333,40.14,86,1,5.68,RU,clear sky
4,4,Hithadhoo,-0.6000,73.0833,83.59,70,100,18.12,MV,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp =float(input("What is the minimum temperature you would like for your trip? " ))
max_temp =float(input("What is the maximum temperature you would like for your trip? " ))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
4,4,Hithadhoo,-0.6000,73.0833,83.59,70,100,18.12,MV,overcast clouds
6,6,Kavaratti,10.5669,72.6420,83.98,76,100,18.25,IN,light rain
7,7,Beyneu,45.3167,55.2000,78.57,51,96,20.60,KZ,overcast clouds
16,16,Batticaloa,7.7102,81.6924,82.83,73,100,0.65,LK,overcast clouds
18,18,La Libertad,-2.2333,-80.9000,76.66,73,75,10.63,EC,broken clouds
28,28,Bud,39.4470,-86.1758,76.73,84,100,6.04,US,moderate rain
33,33,Springfield,37.2153,-93.2982,78.53,58,20,4.00,US,few clouds
34,34,Tainan,22.9908,120.2133,89.15,78,75,5.75,TW,broken clouds
35,35,Garden City,42.3256,-83.3310,78.89,66,75,1.99,US,broken clouds
38,38,Pailon,-17.6500,-62.7500,85.68,41,100,10.49,BO,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
# preferred_cities_df.count()

Unnamed: 0             0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# dropped_df = preferred_cities_df.dropna()
# dropped_df.count()

Unnamed: 0             262
City                   262
Lat                    262
Lng                    262
Max Temp               262
Humidity               262
Cloudiness             262
Wind Speed             262
Country                262
Current Description    262
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = dropped_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hithadhoo,MV,83.59,overcast clouds,-0.6000,73.0833,
6,Kavaratti,IN,83.98,light rain,10.5669,72.6420,
7,Beyneu,KZ,78.57,overcast clouds,45.3167,55.2000,
16,Batticaloa,LK,82.83,overcast clouds,7.7102,81.6924,
18,La Libertad,EC,76.66,broken clouds,-2.2333,-80.9000,
28,Bud,US,76.73,moderate rain,39.4470,-86.1758,
33,Springfield,US,78.53,few clouds,37.2153,-93.2982,
34,Tainan,TW,89.15,broken clouds,22.9908,120.2133,
35,Garden City,US,78.89,broken clouds,42.3256,-83.3310,
38,Pailon,BO,85.68,overcast clouds,-17.6500,-62.7500,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row ["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel added to DF")
    except (IndexError):
            print("Hotel not found... skipping.")
        

Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel not found... skipping.
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel not found... skipping.
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel 

In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()


City                   262
Country                262
Max Temp               262
Current Description    262
Lat                    262
Lng                    262
Hotel Name             262
dtype: int64

In [34]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))